In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
%matplotlib inline
os.getcwd()

In [ ]:
print(tf.__version__)

In [ ]:
image_x = 640
image_y = 360

path = {
    'topcam_folder':os.path.join('dataset', 'topcam'),
    'sidecam_folder':os.path.join('dataset', 'sidecam'),
    'stickcam_folder':os.path.join('dataset', 'stickcam'),
    'checkpoint_path': os.path.join('Model', 'my_ssd_mobnet4')
}

file = {
    'topcam_removeBG_model':os.path.join('Model', 'RemoveBottomBackground2.h5'),
    'sidecam_removeBG_model':os.path.join('Model', 'RemoveBackgroundVer9.h5'),
    'bottomcam_removeBG_model':os.path.join('Model', 'RemoveStickBackground001.h5'),
    'pipeline_config':os.path.join(path['checkpoint_path'], 'pipeline.config'),
    'label_map':os.path.join('annotations', 'label_map.pbtxt')
}

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(file['label_map'])
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(file['pipeline_config'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(path['checkpoint_path'], 'ckpt-201')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


# Top Camera Function

In [ ]:
def Topcam_RemoveBG(image):
    ##### return image with INPUT shape #####
    src = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    [hight, width, _] = src.shape
    src_resize = cv.resize(src, (image_x, image_y))
    src_resize = src_resize / 255.0
    
    model = tf.keras.models.load_model(file['topcam_removeBG_model'])
    predict = model.predict(np.array([src_resize]))
    predict = predict.reshape(image_y, image_x)
    
    ret, predict = cv.threshold(predict, 0.2, 1.0, cv.THRESH_BINARY)
    
    predict = cv.resize(predict, (width, hight)).reshape(hight, width, 1)
    res = np.multiply(src/255., np.repeat(predict, 3, axis = 2))
    res = res * 255
    output = res.astype(np.uint8)

    return output

def Bottom_detection(image):
    ##### return image with MODEL shape #####
    src = cv.resize(image, (image_x, image_y))
    [hight,width,_] = src.shape
    image_np = np.array(src).astype(np.uint8)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections
    
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates = True,
                max_boxes_to_draw = 2,
                min_score_thresh = (0.18),
                agnostic_mode = False)
    

    if(detections['detection_scores'][0] >= 0.18):
        detected = True
    else:
        detected = False
    
    ymin = np.array(detections['detection_boxes'][0][0] * hight)
    xmin = np.array(detections['detection_boxes'][0][1] * width)
    ymax = np.array(detections['detection_boxes'][0][2] * hight)
    xmax = np.array(detections['detection_boxes'][0][3] * width)
    center_x = ((xmax - xmin)/2) + xmin
    center_y = ((ymax - ymin)/2) + ymin
    
    ##### Distance #####
    x_distance = center_x - 316 
    y_distance = center_y - 157
    
    ##### Angle on X Axis ##### 
    ##### NOT USE #####
    topcam_angle = (math.atan2((-y_distance), (x_distance)))
    topcam_angle = topcam_angle*180/math.pi
    
    ##### X Axis Line #####
    cv.line(image_np_with_detections, (316, 0), (316, 360), (0, 255, 0), 2)
    
    ##### Y Axis Line #####
    cv.line(image_np_with_detections, (0, 157), (640, 157), (255, 0, 0), 2)
    
    ##### Connect Line #####
    cv.line(image_np_with_detections, (316, 157) , (int(center_x), int(center_y)), (0, 0, 255), 2)
    
    return image_np_with_detections, x_distance, y_distance, detected

# Side Camera Function

In [ ]:
def Sidecam_RemoveBG(image):
    ##### return image with INPUT shape #####
    src = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    [hight, width, _] = src.shape
    src_resize = cv.resize(src, (256, 144))
    src_resize = src_resize/255.0
    
    model = tf.keras.models.load_model(file['sidecam_removeBG_model'])
    predict = model.predict(np.array([src_resize]))
    predict = predict.reshape(144, 256)
    
    ret, predict = cv.threshold(predict, 0.59, 1.0, cv.THRESH_BINARY)
    
    predict = cv.resize(predict, (width, hight)).reshape(hight, width, 1)
    #plt.imshow(predict, cmap = 'gray')
    #plt.show()
    res = np.multiply(src/255., np.repeat(predict, 3, axis = 2))
    res = res * 255
    output = res.astype(np.uint8)
    
    return output

def Max_Area_Contour(image):
    ##### return max area contour #####
    image = image.astype('uint8')
    if(len(image.shape) != 3):
        src_gray = image
    else:
        src_gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
        
    src_gray = cv.medianBlur(src_gray, 15)
    
    _, thresh = cv.threshold(src_gray, 30, 255, cv.THRESH_BINARY)
    contours , hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    max_area_countour = max(contours, key = cv.contourArea)
    
    return max_area_countour

def Point_Cam_Frame(c, x_distance, y_distance):
    index = (np.where(c[:,0,0] == max(c[:,0,0])))[0][0]
    heightest_point = c[index][0]
    heightest_x = heightest_point[0]
    heightest_y = heightest_point[1]
    ##### Z distance is 1920x1080 scale
    z_distance = math.sqrt( ((heightest_x - 382)**2) + ((heightest_y - 530)**2) )
    
    ##### rescale to 640x360 #####
    z_distance = z_distance/3
    
    ##### Rotation Matrix #####
    theta = np.radians(145)
    [cos, sin] = np.cos(theta), np.sin(theta)
    [bottom_x, bottom_y, bottom_z] = [((x_distance*cos) - (-y_distance*sin)), ((x_distance*sin) + (-y_distance*cos)), (z_distance)]
    
    return z_distance, bottom_x, bottom_y, bottom_z

def Crop_Image_From_Contour(image):
    c = Max_Area_Contour(image)
    x, y ,w, h = cv.boundingRect(c)
    #cv.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    crop_img = image[0:(530*2), x:x+w]

    return crop_img

# Stick Camera Function

In [ ]:
def Stickcam_RemoveBG(image):
    ##### return image with INPUT shape #####
    src = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    [hight, width, _] = src.shape
    src_resize = cv.resize(src, (256, 144))
    src_resize = src_resize/255.0
    
    model = tf.keras.models.load_model(file['bottomcam_removeBG_model'])
    predict = model.predict(np.array([src_resize]))
    predict = predict.reshape(144, 256)
    
    ret, predict = cv.threshold(predict, 0.4, 1.0, cv.THRESH_BINARY)
    
    predict = cv.resize(predict, (width, hight)).reshape(hight, width, 1)
    #plt.imshow(predict, cmap = 'gray')
    #plt.show()
    res = np.multiply(src/255., np.repeat(predict, 3, axis = 2))
    res = res * 255
    output = res.astype(np.uint8)
    
    return output

# ALL SCRIPT

In [ ]:
################################ RANDOM PICTURE #########################################################
topcam_folder = next(os.walk(path['topcam_folder']))[1]
sidecam_folder = next(os.walk(path['sidecam_folder']))[1]
random_folder_number = random.randint(0, (len(sidecam_folder)-1))
topcam_folder_path = os.path.join(path['topcam_folder'], topcam_folder[random_folder_number])
sidecam_folder_path = os.path.join(path['sidecam_folder'], sidecam_folder[random_folder_number])

topcam_pic_name = next(os.walk(topcam_folder_path))[2]
sidecam_pic_name = next(os.walk(sidecam_folder_path))[2]

########################################################################################################

for i in range(len(topcam_pic_name)):
    topcam_picture_path = os.path.join(topcam_folder_path, topcam_pic_name[i])
    topcam_image = cv.imread(topcam_picture_path, cv.IMREAD_COLOR)
    sidecam_picture_path = os.path.join(sidecam_folder_path, sidecam_pic_name[i])
    sidecam_image = cv.imread(sidecam_picture_path, cv.IMREAD_COLOR)
    
    ##### Top Camera Part #####
    topcam_removeBG = Topcam_RemoveBG(topcam_image)
    buttom_detection, x_distance, y_distance, detected = Bottom_detection(topcam_removeBG)
    
    ##### Side Camera Part #####
    if(detected):
        sidecam_removeBG = Sidecam_RemoveBG(sidecam_image)
        c = Max_Area_Contour(sidecam_removeBG)
        z_distance, bottom_x, bottom_y, bottom_z = Point_Cam_Frame(c, x_distance, y_distance)
        durian_angle = (math.atan2(((int(177 + bottom_x)*3) - 530), ((int(127 + bottom_z)*3) - 382))) * (180/math.pi)
    
        rotate_matrix = cv.getRotationMatrix2D(center = (382, 530), angle= int(durian_angle), scale=1)
        rotated_image = cv.warpAffine(src = sidecam_removeBG, M = rotate_matrix, dsize = (sidecam_removeBG.shape[1], sidecam_removeBG.shape[0]))
        
        crop_rotate_image = Crop_Image_From_Contour(rotated_image)
        mirror = np.flipud(crop_rotate_image)

        intersec_image = cv.bitwise_and(cv.cvtColor(crop_rotate_image, cv.COLOR_RGB2GRAY), cv.cvtColor(mirror, cv.COLOR_RGB2GRAY))
        intersec_image[intersec_image != 0] = 255
        
        union_image = cv.bitwise_or(cv.cvtColor(crop_rotate_image, cv.COLOR_RGB2GRAY), cv.cvtColor(mirror, cv.COLOR_RGB2GRAY))
        union_image[union_image != 0] = 255

        intersec_area = cv.contourArea(Max_Area_Contour(intersec_image))
        union_area = cv.contourArea(Max_Area_Contour(union_image))

        cv.drawContours(crop_rotate_image, Max_Area_Contour(intersec_image), -1, (255, 0, 0), 3)

        inersec_percentage = (intersec_area / union_area)*100
        print('Intersection Percentage: ' + str(round(inersec_percentage)) + '%')

        ##### X Axis On Side Camera #####
        cv.line(sidecam_removeBG, (382, 0), (382, 1080), (255, 0, 0), 3)
        cv.line(rotated_image, (382, 0), (382, 1080), (255, 0, 0), 3)
        ##### Y Axis On Side Camera #####
        cv.line(sidecam_removeBG, (0, 530), (1920, 530), (0, 0, 255), 3)
        cv.line(rotated_image, (0, 530), (1920, 530), (0, 0, 255), 3)
        ##### Orientation line #####
        cv.line(sidecam_removeBG, (382, 530), (int(127 + bottom_z)*3, int(177 + bottom_x)*3), (255, 255, 255), 5)

        #plt.imshow(buttom_detection)
        #plt.show()
        #print('X distance: ' + str(x_distance))
        #print('Y distance: ' + str(y_distance))
        #print('Z distance: ' + str(z_distance))
        #print('Z point: ' + str(bottom_z))
        #print('X point: ' + str(bottom_x))

        #plt.imshow(sidecam_removeBG)
        plt.imshow(cv.resize(sidecam_removeBG, (image_x, image_y)))
        #plt.imshow(cv.rotate(sidecam_removeBG, cv.ROTATE_90_COUNTERCLOCKWISE))
        #plt.imshow(cv.rotate(cv.resize(sidecam_removeBG, (image_x, image_y)), cv.ROTATE_90_COUNTERCLOCKWISE))
        plt.show()

        plt.imshow(cv.hconcat([cv.rotate(crop_rotate_image, cv.ROTATE_90_COUNTERCLOCKWISE), cv.rotate(mirror, cv.ROTATE_90_COUNTERCLOCKWISE)]))
        plt.show()
        
        plt.imshow(cv.hconcat([cv.rotate(intersec_image, cv.ROTATE_90_COUNTERCLOCKWISE), cv.rotate(union_image, cv.ROTATE_90_COUNTERCLOCKWISE)]), cmap = 'gray')
        plt.show()

    else:
        print('CANNOT FIND BOTTOM OF THIS OBJECT')

# RUN ALL

In [ ]:
topcam_folder = next(os.walk(path['topcam_folder']))[1]

for i in topcam_folder:
    topcam_folder_path = os.path.join(path['topcam_folder'], str(i))
    sidecam_folder_path = os.path.join(path['sidecam_folder'], str(i))
    stickcam_folder_path = os.path.join(path['stickcam_folder'], str(i))

    topcam_pic_name = next(os.walk(topcam_folder_path))[2]
    sidecam_pic_name = next(os.walk(sidecam_folder_path))[2]
    stickcam_pic_name = next(os.walk(stickcam_folder_path))[2]


    for j in range(len(sidecam_pic_name)):
        topcam_pic_path = os.path.join(topcam_folder_path, topcam_pic_name[j])
        topcam_image = cv.imread(topcam_pic_path, cv.IMREAD_COLOR)
        sidecam_pic_path = os.path.join(sidecam_folder_path, sidecam_pic_name[j])
        sidecam_image = cv.imread(sidecam_pic_path, cv.IMREAD_COLOR)
        stickcam_pic_path = os.path.join(stickcam_folder_path, stickcam_pic_name[j])
        stickcam_image = cv.imread(stickcam_pic_path, cv.IMREAD_COLOR)

        ##### Bottom Camera Part #####
        stickcam_removeBG = Stickcam_RemoveBG(stickcam_image)
        plt.imshow(stickcam_removeBG)
        plt.show()
        
        ##### Top Camera Part #####
        topcam_removeBG = Topcam_RemoveBG(topcam_image)
        buttom_detection, x_distance, y_distance, detected = Bottom_detection(topcam_removeBG)

        ##### Side Camera Part #####
        if(detected):
            sidecam_removeBG = Sidecam_RemoveBG(sidecam_image)
            c = Max_Area_Contour(sidecam_removeBG)
            z_distance, bottom_x, bottom_y, bottom_z = Point_Cam_Frame(c, x_distance, y_distance)
            durian_angle = (math.atan2(((int(177 + bottom_x)*3) - 530), ((int(127 + bottom_z)*3) - 382))) * (180/math.pi)

            rotate_matrix = cv.getRotationMatrix2D(center = (382, 530), angle= int(durian_angle), scale=1)
            rotated_image = cv.warpAffine(src = sidecam_removeBG, M = rotate_matrix, dsize = (sidecam_removeBG.shape[1], sidecam_removeBG.shape[0]))

            crop_rotate_image = Crop_Image_From_Contour(rotated_image)
            mirror = np.flipud(crop_rotate_image)

            intersec_image = cv.bitwise_and(cv.cvtColor(crop_rotate_image, cv.COLOR_RGB2GRAY), cv.cvtColor(mirror, cv.COLOR_RGB2GRAY))
            intersec_image[intersec_image != 0] = 255

            union_image = cv.bitwise_or(cv.cvtColor(crop_rotate_image, cv.COLOR_RGB2GRAY), cv.cvtColor(mirror, cv.COLOR_RGB2GRAY))
            union_image[union_image != 0] = 255

            intersec_area = cv.contourArea(Max_Area_Contour(intersec_image))
            union_area = cv.contourArea(Max_Area_Contour(union_image))

            cv.drawContours(crop_rotate_image, Max_Area_Contour(intersec_image), -1, (255, 0, 0), 3)

            inersec_percentage = (intersec_area / union_area)*100
            print('Intersection Percentage: ' + str(round(inersec_percentage)) + '%')

            ##### X Axis On Side Camera #####
            cv.line(sidecam_image, (382, 0), (382, 1080), (255, 0, 0), 3)
            cv.line(sidecam_removeBG, (382, 0), (382, 1080), (255, 0, 0), 3)
            cv.line(rotated_image, (382, 0), (382, 1080), (255, 0, 0), 3)
            ##### Y Axis On Side Camera #####
            cv.line(sidecam_image, (0, 530), (1920, 530), (0, 0, 255), 3)
            cv.line(sidecam_removeBG, (0, 530), (1920, 530), (0, 0, 255), 3)
            cv.line(rotated_image, (0, 530), (1920, 530), (0, 0, 255), 3)
            ##### Orientation line #####
            cv.line(sidecam_image, (382, 530), (int(127 + bottom_z)*3, int(177 + bottom_x)*3), (255, 255, 255), 5)
            cv.line(sidecam_removeBG, (382, 530), (int(127 + bottom_z)*3, int(177 + bottom_x)*3), (255, 255, 255), 5)

            #plt.imshow(buttom_detection)
            #plt.show()
            #print('X distance: ' + str(x_distance))
            #print('Y distance: ' + str(y_distance))
            #print('Z distance: ' + str(z_distance))
            #print('Z point: ' + str(bottom_z))
            #print('X point: ' + str(bottom_x))

            #plt.imshow(cv.cvtColor(sidecam_image, cv.COLOR_RGB2BGR))
            #plt.imshow(cv.resize(cv.cvtColor(sidecam_image, cv.COLOR_RGB2BGR), (image_x, image_y)))
            #plt.imshow(cv.rotate(sidecam_removeBG, cv.ROTATE_90_COUNTERCLOCKWISE))
            #plt.imshow(cv.rotate(cv.resize(sidecam_removeBG, (image_x, image_y)), cv.ROTATE_90_COUNTERCLOCKWISE))
            #plt.show()

            #plt.imshow(cv.hconcat([cv.rotate(crop_rotate_image, cv.ROTATE_90_COUNTERCLOCKWISE), cv.rotate(mirror, cv.ROTATE_90_COUNTERCLOCKWISE)]))
            #plt.show()
            #plt.imshow(cv.hconcat([cv.rotate(intersec_image, cv.ROTATE_90_COUNTERCLOCKWISE), cv.rotate(union_image, cv.ROTATE_90_COUNTERCLOCKWISE)]), cmap = 'gray')
            #plt.show()

        else:
            print('CANNOT FIND BOTTOM OF THIS OBJECT')
        

# Change file path

In [ ]:
import os
import shutil
import math

pic_name = next(os.walk('Train'))[2]
pic_num = len(pic_name)
test_num = math.floor(pic_num*30/100)
val_num = math.floor((pic_num - test_num)*30/100)
train_num = pic_num - val_num - test_num

random_picture_name = random.sample(pic_name, test_num)
print(random_picture_name)
for i in range(len(random_picture_name)):
    #shutil.move(os.path.join('Train', random_picture_name[i]), os.path.join('Test', random_picture_name[i]))
    break

pic_name = next(os.walk('Train'))[2]
random_picture_name = random.sample(pic_name, val_num)
                                     
for j in range(len(random_picture_name)):
    #shutil.move(os.path.join('Train', random_picture_name[j]), os.path.join('Val', random_picture_name[j]))
    break

In [ ]:
stickcam_folder = next(os.walk(path['stickcam_folder']))[1]
center_x = 960
center_y = 540

for i in stickcam_folder:
    stickcam_folder_path = os.path.join(path['stickcam_folder'], str(i))
    stickcam_pic_name = next(os.walk(stickcam_folder_path))[2]


    for j in range(len(stickcam_pic_name)):
        bottomcam_pic_path = os.path.join(stickcam_folder_path, stickcam_pic_name[j])
        bottomcam_image = cv.imread(bottomcam_pic_path, cv.IMREAD_COLOR)

        bottomcam_image = bottomcam_image[center_y - 540:center_y + 540 ,center_x - 540:center_x + 540]
        bottomcam_image = cv.resize(bottomcam_image, (320, 320))
        output_path = os.path.join('Train', stickcam_pic_name[j])
        #cv.imwrite(output_path, bottomcam_image)
